# Part 2 Get Foursquare Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import requests 
import io
import ibm_boto3
from pandas.io.json import json_normalize

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
#Import Metro dataframe
obj = resource.Object(bucket_name=bucket, key='MetroData.json').get()
df = pd.read_json(io.BytesIO(obj['Body'].read()))
df.head()

,Metropolitan,Country,Continent,Latitude,Longitude
0,Tokyo,Japan,Asia,35.682839,139.759455
1,Delhi,India,Asia,28.651718,77.221939
10,Mexico City,Mexico,North America,19.432601,-99.133342
11,São Paulo,Brazil,South America,-23.550651,-46.633382
12,Lagos,Nigeria,Africa,6.455057,3.394179


In [4]:
# The code was removed by Watson Studio for sharing.

Credentails set


In [5]:
#Function gets venue data for a location and returns dataframe with info from API request
def get_data(lat,lng):
    LIMIT = 200
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}&section=topPicks&time=any&day=any'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        LIMIT)
            
    # make the GET request
    results = requests.get(url).json()
    #results
    #print(results)
    try:
        df_4s = json_normalize(results['response']['groups'][0]['items'])
    except:
        results = requests.get(url).json()
        df_4s = json_normalize(results['response']['groups'][0]['items'])
    return df_4s

In [6]:
#function gets the category of the venues and returns a list of categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
#function calculates the top venues for the location and returns a dataframe with top 10 venues
def get_venues(df_4s):
    # filter columns
    filtered_columns = ['venue.name', 'venue.categories'] + [col for col in df_4s.columns if col.startswith('venue.location.')] + ['venue.id']
    dataframe_filtered = df_4s.loc[:, filtered_columns]
    # filter the category for each row
    dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    # clean columns
    dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
    #Create a dataframe that has venue categories ordered from most frequent to least frequent
    df_New = pd.DataFrame({'Category':dataframe_filtered.categories, 'Name':dataframe_filtered.name})
    df_New = df_New.groupby(['Category'],as_index = False).count()
    df_New.sort_values(by=['Name'], ascending=False, inplace=True)
    df_New.reset_index(drop = True, inplace=True)
    return df_New.head(10)

In [8]:
#Function creates a new row for the table 
def create_row(i):
    new_row = []
    #Enter location infomation into the row
    for j in range(5):
        new_row.append(df.iloc[i,j])
    #Enter top venue categories into the row
    for k in range(10):
        try:
            new_row.append(df_New.iloc[k,0])
        except:
            new_row.append(np.NAN)
    return new_row

In [9]:
#Create the venue dataframe
#create list of column names
colNames = ['Metropolitan','Country', 'Continent','Latitude', 'Longitude']
for i in range(10):
    colNames.append('Category ' + str(i+1))

#Create the dataframe using rows in df
df_cat = pd.DataFrame(columns= colNames)
for n in range(len(df.index)):
    lat = df.iloc[n,3]
    lng = df.iloc[n,4]
    df_New = get_venues(get_data(lat,lng))
    df_cat = df_cat.append(pd.DataFrame([create_row(n)], columns= colNames), ignore_index=True)

In [10]:
df_cat

,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10
0,Tokyo,Japan,Asia,35.682839,139.759455,Café,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Lounge,Garden,French Restaurant,Chinese Restaurant,Dessert Shop,Thai Restaurant
1,Delhi,India,Asia,28.651718,77.221939,Indian Restaurant,Bar,Café,Flea Market,Lounge,Asian Restaurant,Ice Cream Shop,South Indian Restaurant,Portuguese Restaurant,Plaza
2,Mexico City,Mexico,North America,19.432601,-99.133342,Mexican Restaurant,Art Museum,Museum,Arts & Crafts Store,Bar,Ice Cream Shop,Boutique,Bakery,Restaurant,Café
3,São Paulo,Brazil,South America,-23.550651,-46.633382,Brazilian Restaurant,Japanese Restaurant,Café,Bakery,Snack Place,Bookstore,Plaza,Cosmetics Shop,Chinese Restaurant,Art Gallery
4,Lagos,Nigeria,Africa,6.455057,3.394179,Lounge,African Restaurant,Café,Bar,Italian Restaurant,Pizza Place,Art Gallery,Hotel,Shopping Mall,Nightclub
5,Mumbai,India,Asia,18.938771,72.835335,Indian Restaurant,Café,Seafood Restaurant,Bakery,Bar,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Bookstore,Ice Cream Shop
6,Cairo,Egypt,Africa,30.048819,31.243666,Café,Lounge,Middle Eastern Restaurant,Italian Restaurant,Theater,Egyptian Restaurant,Lebanese Restaurant,Coffee Shop,Hotel Bar,Boat or Ferry
7,Kyoto,Japan,Asia,35.021041,135.755607,Café,Japanese Restaurant,Bakery,Sake Bar,Dessert Shop,Palace,Ramen Restaurant,Soba Restaurant,Coffee Shop,Art Gallery
8,Wuhan,China,Asia,30.595105,114.299935,Shopping Mall,Park,Chinese Restaurant,Department Store,Asian Restaurant,Beer Garden,Coffee Shop,Hubei Restaurant,Movie Theater,Pedestrian Plaza
9,Los Angeles,United States,North America,34.053691,-118.242767,Sushi Restaurant,Bar,Coffee Shop,Plaza,Mexican Restaurant,Japanese Restaurant,Ice Cream Shop,Seafood Restaurant,Cocktail Bar,Museum


In [11]:
#Save dataframe to json file
resource.Bucket(name=bucket).put_object(Key='VenueData.json', Body=df_cat.to_json())
print('Saved')

Saved
